In [1]:
# Assignment 2 
import threading
import time
import timeit
import pynq.lib.rgbled as rgbled
from enum import Enum
from pynq.overlays.base import BaseOverlay

base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"
//#include "stdio.h"

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    read_gpio(pin);
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}


//1. Code to reset all pins
void reset_all_gpio_pins() {
    pyprintf("reset_all_gpio_pins");
     for(int i=0;i<8;i++) {
        write_gpio(i,0);
    }
}

In [3]:
shift_pin=0  # shall be 4 if you want to sue the pins 4-6
blue = 1 + shift_pin
green = 2 +shift_pin
red = 3 + shift_pin
btn0 = base.buttons[0]
led4 = rgbled.RGBLED(4)


In [4]:
class Phil_state(Enum):
    STARVING = 1 #default
    EATING = 2
    NAPPING = 3
    
    
    
def blink(t, d, n):
    rbgled=0
    '''
    Function to blink the LEDs
    Params:
      t: number of times to blink the LED
      d: duration (in seconds) for the LED to be on/off
      n: index of the LED (0 to 3)
    '''
    print(f"called blink: LED-n={n}= t/num/range={t}Xduration/d={d}secs")
    for i in range(t):
        if n<4:
            base.leds[n].toggle()
            time.sleep(d)
            
        else  :
            '''write_gpio(green,1)
            time.sleep(d)
            write_gpio(green,0)
            '''
            if(rbgled==0):
                led4.write(0x7)
                rbgled=1
            else :
                led4.write(0x0)
                rbgled=0
            time.sleep(d)  
        if btn0.read() != 0:
            print("blinking: Pressed button 0 . STOP.lights. OFF")
            led4.write(0x0)
            base.leds[n].off()
            #system.exit()
            return
            
    if n<4 :
        base.leds[n].off()
    else :
        #write_gpio(green,0) 
        led4.write(0x0)
        
def check_if_forks_avail(num,lock,left_fork,right_fork):
    #for i in range(5):
        #print(f"fork{i}=is locked?{lock[i].locked()}")
    #print(f"check_if_forks_avail called : phil:{num},right_f{right_fork}={lock[right_fork].locked()},left_f({left_fork})={lock[left_fork].locked()}")
    if ((not lock[left_fork].locked()) and (not lock[right_fork].locked())):
        #print("both right left fork are availble for philospher",num)
        return 1        
    else :
        '''
        print("DIDNOT get both left fork and right fork for philospher",num)
        if (lock[left_fork].locked()):
            print("DIDNOT get left fork for philospher",num)
        if (lock[right_fork].locked()):
            print("DIDNOT get right fork for philospher",num)
        '''    
        return 0

In [5]:
TOTAL_TIME_TO_EAT_MEAL=10
TIME_EACH_GET_FORK=5
TOTAL_TIME_TO_NAP=TOTAL_TIME_TO_EAT_MEAL*1

def philospher_t(lock,indx,sleep_time) : 
    fork_r = []
    total_ate=0
    total_slept=0
    state=Phil_state.STARVING
    print(f"in phil{indx} called at time={{timeit.default_timer()}}: sleep_time={sleep_time} ")
    # 0 5 4 3 2 1 0
    #required fork would be fork on right=num and  left=num+1, if right=num==4 left=0 
    right_fork=indx
    if(indx==4):
        left_fork=0
    else:
        left_fork=indx+1
    print("in phil : left fork: right fork is ",indx,right_fork,left_fork)
        
    cnt=0   
    while True :
    #while cnt<10:    
        cnt=cnt+1
        print(f"time={timeit.default_timer()} Phil={indx} Entered while loop cnt={cnt} state={state}")
        if(state==Phil_state.STARVING):
            if(check_if_forks_avail(indx,lock,left_fork,right_fork)):
                lock[left_fork].acquire(True)
                lock[right_fork].acquire(True)
                print("Acquired forks for philospher",indx)
                state=Phil_state.EATING
            #else :
                #print(f"STARVING : Phil{num} I dont have both locks. Stay starving")
  
        elif(state==Phil_state.EATING):
            print(f"philospher={indx} EATING: already ate still now={total_ate}")
            #if(check_if_forks_avail(indx,lock,left_fork,right_fork)):
            if (True) :   
                #time.sleep(TIME_EACH_GET_FORK)
                #TOTAL_TIME_TO_EAT_MEAL
                total_ate=total_ate+TIME_EACH_GET_FORK
                d=TIME_EACH_GET_FORK/10
                num_blink=TIME_EACH_GET_FORK/10
                #blink(num_blink,d,indx)
                blink(50,0.2,indx)
                lock[left_fork].release()
                lock[right_fork].release()
                #blink(time,duration,num)
                #t: number of times to blink the LED
                #d: duration (in seconds) for the LED to be on/off
                #n: index of the LED (0 to 3)
                print("philospher DONE EATING: ",indx)
                state=Phil_state.NAPPING
            else :
                    state=Phil_state.STARVING
                    print("philospher was EATING: ERROR but no forks so move to STARVING: ",indx,total_ate)
        elif (state==Phil_state.NAPPING):
            print("philospher NAPPING: already slept this much: ",indx,total_slept)
            #total_slept=total_slept+TIME_TO_NAP_EACH_TIME
            #d=TIME_TO_NAP_EACH_TIME/5
            d=TOTAL_TIME_TO_NAP/2
            num_blink=TOTAL_TIME_TO_NAP/d
            #blink(num_blink,d,indx)
            blink(10,1,indx)
            #print(f"philospher{indx} NAPPING: total_slept till now {total_slept}: ")
            print("philospher DONE EATING: ",indx)
            state=Phil_state.STARVING
        if btn0.read() != 0:
            print("Pressed button 0 . STOP.lights. OFF")
            #system.exit()
            return
        print(f"phil thread{indx} calling sleep at time={timeit.default_timer()}")
        #time.sleep(sleep_time)

In [6]:
# Initialize and launch the threads
# A2.1
threads = []
lock = []
#sleep_time=0.001 //straving threads print too much
sleep_time=0.5
write_gpio(green,0)
write_gpio(red,0)
for i in range(0,5,1):
    print("create lock num : ",i)
    lock.append(threading.Lock())
for i in range(0,5,1):     #Try with 1Philospher
    t = threading.Thread(target=philospher_t, args=(lock, i,sleep_time))
    threads.append(t)
    t.start()
    print("started Thread : ",i)

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))

create lock num :  0
create lock num :  1
create lock num :  2
create lock num :  3
create lock num :  4
in phil0 called at time={timeit.default_timer()}: sleep_time=0.5 
in phil : left fork: right fork is  0 0 1
time=1129.192255494 Phil=0 Entered while loop cnt=1 state=Phil_state.STARVING
Acquired forks for philospher 0
phil thread0 calling sleep at time=1129.192747679
time=1129.192866183 Phil=0 Entered while loop cnt=2 state=Phil_state.EATING
philospher=0 EATING: already ate still now=0
called blink: LED-n=0= t/num/range=50Xduration/d=0.2secs
started Thread :  0
in phil1 called at time={timeit.default_timer()}: sleep_time=0.5 
in phil : left fork: right fork is  1 1 2
time=1129.196015814 Phil=1 Entered while loop cnt=1 state=Phil_state.STARVING
phil thread1 calling sleep at time=1129.196351697
time=1129.19647163 Phil=1 Entered while loop cnt=2 state=Phil_state.STARVING
phil thread1 calling sleep at time=1129.196709113
time=1129.196826688 Phil=1 Entered while loop cnt=3 state=Phil_sta

/tmp/ipykernel_2418/3855363165.py:19: DeprecationWarning: getName() is deprecated, get the name attribute instead
  name = t.getName()


time=1129.386151949 Phil=3 Entered while loop cnt=105 state=Phil_state.STARVING
phil thread3 calling sleep at time=1129.391859555
time=1129.391972989 Phil=3 Entered while loop cnt=106 state=Phil_state.STARVING
phil thread3 calling sleep at time=1129.392175518
time=1129.392262306 Phil=3 Entered while loop cnt=107 state=Phil_state.STARVING
phil thread3 calling sleep at time=1129.392439629
time=1129.392525605 Phil=3 Entered while loop cnt=108 state=Phil_state.STARVING
phil thread3 calling sleep at time=1129.392697642
time=1129.392782454 Phil=3 Entered while loop cnt=109 state=Phil_state.STARVING
phil thread3 calling sleep at time=1129.392954226
time=1129.395418602 Phil=1 Entered while loop cnt=170 state=Phil_state.STARVING
time=1129.396579303 Phil=4 Entered while loop cnt=104 state=Phil_state.STARVING
phil thread4 calling sleep at time=1129.396910546
time=1129.397046171 Phil=4 Entered while loop cnt=105 state=Phil_state.STARVING
phil thread4 calling sleep at time=1129.397294774
time=1129.

Exception in thread Thread-9 (philospher_t):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_2418/2903854545.py", line 44, in philospher_t
  File "/tmp/ipykernel_2418/1081366671.py", line 38, in blink
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/lib/axigpio.py", line 197, in __getitem__
    raise IndexError()
IndexError


time=1144.435595244 Phil=3 Entered while loop cnt=26591 state=Phil_state.STARVING
phil thread3 calling sleep at time=1144.445000308
time=1144.445115145 Phil=3 Entered while loop cnt=26592 state=Phil_state.STARVING
phil thread3 calling sleep at time=1144.445317299
time=1144.445405465 Phil=3 Entered while loop cnt=26593 state=Phil_state.STARVING
phil thread3 calling sleep at time=1144.445576105
time=1144.445659201 Phil=3 Entered while loop cnt=26594 state=Phil_state.STARVING
phil thread3 calling sleep at time=1144.445830382
time=1144.445916825 Phil=3 Entered while loop cnt=26595 state=Phil_state.STARVING
phil thread3 calling sleep at time=1144.446089742
time=1144.446174887 Phil=3 Entered while loop cnt=26596 state=Phil_state.STARVING
phil thread3 calling sleep at time=1144.44634373
time=1144.446429158 Phil=3 Entered while loop cnt=26597 state=Phil_state.STARVING
phil thread3 calling sleep at time=1144.44659773
time=1144.446681711 Phil=3 Entered while loop cnt=26598 state=Phil_state.STARV

In [7]:
help(rgbled)

Help on module pynq.lib.rgbled in pynq.lib:

NAME
    pynq.lib.rgbled

DESCRIPTION
    #   Copyright (c) 2016, Xilinx, Inc.
    #   SPDX-License-Identifier: BSD-3-Clause

CLASSES
    builtins.object
        RGBLED
    
    class RGBLED(builtins.object)
     |  RGBLED(index, ip_name='rgbleds_gpio', start_index=inf)
     |  
     |  This class controls the onboard RGB LEDs.
     |  
     |  Attributes
     |  ----------
     |  index : int
     |      The index of the RGB LED. Can be an arbitrary value.
     |  _mmio : MMIO
     |      Shared memory map for the RGBLED GPIO controller.
     |  _rgbleds_val : int
     |      Global value of the RGBLED GPIO pins.
     |  _rgbleds_start_index : int
     |      Global value representing the lowest index for RGB LEDs
     |  
     |  Methods defined here:
     |  
     |  __init__(self, index, ip_name='rgbleds_gpio', start_index=inf)
     |      Create a new RGB LED object.
     |      
     |      Parameters
     |      ----------
     |     